In [ ]:
pip install -U efficientnet

     |████████████████████████████████| 51kB 5.0MB/s 


In [ ]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
model = EfficientNetB0(include_top=False, weights=None, input_shape=(32,32,3))
# summarize the model
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 32, 32, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 33, 33, 3)    0           normalization[0][0]              
_____________________________________________________________________________________

In [ ]:
len(model.layers)

237

In [ ]:
model.layers[-1].output

<KerasTensor: shape=(None, 1, 1, 1280) dtype=float32 (created by layer 'top_activation')>

In [ ]:
# add new classifier layers for cifar10 classification
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

flat = Flatten()(model.layers[-1].output)
class1 = Dense(512, activation='relu')(flat)

prediction = Dense(10, activation='softmax')(class1)

model = Model(inputs=model.inputs, outputs=prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 32, 32, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 33, 33, 3)    0           normalization[0][0]              
______________________________________________________________________________________________

In [ ]:
model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

In [ ]:
import time

start = time.time()

history = model.fit(
    x=x_train,
    y=y_train,
    epochs=100,
    verbose=1,
    validation_data=(x_test, y_test),
    batch_size=128
)

epochs = [20, 30, 40, 50, 60, 70, 80, 90, 100]
acc_epoch = []

for epoch in epochs:
  acc_epoch.append(history.history.get('val_accuracy')[epoch-1])
  
end = time.time()

Epoch 1/100
391/391 [==============================] - 28s 72ms/step - loss: 2.0293 - accuracy: 0.2421 - val_loss: 2.3415 - val_accuracy: 0.1589
Epoch 2/100
391/391 [==============================] - 26s 66ms/step - loss: 1.6896 - accuracy: 0.3690 - val_loss: 1.5753 - val_accuracy: 0.4165
Epoch 3/100
391/391 [==============================] - 26s 66ms/step - loss: 1.5206 - accuracy: 0.4391 - val_loss: 1.4816 - val_accuracy: 0.4634
Epoch 4/100
391/391 [==============================] - 26s 66ms/step - loss: 1.3955 - accuracy: 0.4907 - val_loss: 1.4212 - val_accuracy: 0.4847
Epoch 5/100
391/391 [==============================] - 26s 66ms/step - loss: 1.2853 - accuracy: 0.5350 - val_loss: 1.3958 - val_accuracy: 0.5036
Epoch 6/100
391/391 [==============================] - 26s 67ms/step - loss: 1.1925 - accuracy: 0.5729 - val_loss: 1.2830 - val_accuracy: 0.5435
Epoch 7/100
391/391 [==============================] - 26s 67ms/step - loss: 1.1052 - accuracy: 0.6064 - val_loss: 1.2785 - val_ac

In [ ]:
acc_epoch

[0.6223999857902527,
 0.6449000239372253,
 0.6570000052452087,
 0.6625000238418579,
 0.6647999882698059,
 0.6769999861717224,
 0.678600013256073,
 0.6632000207901001,
 0.6862000226974487]

In [ ]:
print(end-start)

2607.341417312622


In [ ]:
i = 0
for layer in model.layers:
  if 'add' in layer.name:
    print("Layer number: ", i, ", Name: ", layer.name)
  i += 1

Layer number:  45 , Name:  block2b_add
Layer number:  74 , Name:  block3b_add
Layer number:  103 , Name:  block4b_add
Layer number:  118 , Name:  block4c_add
Layer number:  146 , Name:  block5b_add
Layer number:  161 , Name:  block5c_add
Layer number:  190 , Name:  block6b_add
Layer number:  205 , Name:  block6c_add
Layer number:  220 , Name:  block6d_add


In [ ]:
exits = [45, 74, 103, 118, 146, 161, 190, 205]

In [ ]:
i = 1
for exit in exits:
  print(exit)
  exit_layer = model.layers[exit]
  Model(inputs = model.inputs, outputs = exit_layer.output).save_weights(f"/EffNet/Main_till_exit{i}_weights.h5")
  i += 1

45
74
103
118
146
161
190
205
